# Continuous Run 

This notebook reviews the ```slurm-#######_#.out``` files that are generated when running ```continuousrun.sb```.  Specificall this file parses specific lines in the output file. These include the following examples. The first describes the setup of the run:

```Namespace(input_file='./see-segment/Image_data/Examples/Mushrooms.jpeg', input_mask='./see-segment/Image_data/Examples/Mushrooms_GT.png', seed=64467698, pop_size=10, num_iter=10)```

The others describe each generation's best so far.

```#BEST [0.9278057254150632,  ['XYZ', True, 2, 'Felzenszwalb', 0.9296875, 0.76953125, 0.46875, 0.53125, 0.1953125, 0.3125, 7, 17]]```


# Useful Command

Use the following command on a develompent node to cancel all currently running job arrays without canceling a jupyter ondemand instance:

```scancel `sq | grep _ | grep -v -e TIME | awk '{print $1}'` ```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from pathlib import Path
import re

In [ ]:
slurmoutdir = Path('.')

slurmfiles = slurmoutdir.glob('./*.out')

filelist = []

for file in slurmfiles:
    jobnum = re.findall('-[0-9]*_', str(file))[0][1:-1]
    array = re.findall('_[0-9]*\.', str(file))[0][1:-1]
    item = [str(file), int(jobnum), int(array)]
    filelist.append(item)

df = pd.DataFrame(filelist, columns=['filename', 'jobid', 'arrayid'])
df = df.sort_values(by=['arrayid', 'jobid'])

In [ ]:
def Namespace(input_file='', input_mask='', seed=60, pop_size=0, num_iter=0):
    ns = {}
    ns['input_file'] = input_file;
    ns['input_mask'] = input_mask;   
    ns['seed'] = seed;
    ns['pop_size'] = pop_size;
    ns['num_iter'] = num_iter
    #print(ns)
    return ns

In [ ]:
item = 4

In [ ]:
def parseoutput(filename):
    bestsofar = []
    stats = {}
    times = []
    with open(filename, "r") as f:
        index = 0
        for line in f:
            if 'TRUE_BST' in line:
                vecstr = line[9:]
            if 'BEST' in line:
                vecstr = line[6:]
                x = eval(vecstr)
                x = [index] + x
                bestsofar.append(x)
            if 'Namespace' in line:
                data = line[10:-2].split(',')
                for item in data:
                    name,value = item.strip().split('=')
                    stats[name]=value   
            if 'TIME' in line:
                times.append(float(line.split(' ')[1]))
            #if 'HASH' in line:
                
            index += 1

    return bestsofar, stats, times
    

In [ ]:
examples = {}
iterations = {}
rundata = {}
times = {}
for index, row in df.iterrows():
    if True: #if f"_{item}." in row['filename']:
        bestsofar, stats, thistime = parseoutput(row['filename'])
        #print(f"{row['filename']} - {len(bestsofar)}")
        if not row['arrayid'] in examples:
            examples[row['arrayid']] = bestsofar
            iterations[row['arrayid']] = [ len(examples[row['arrayid']]) ]
            rundata[row['arrayid']] = stats
            times[row['arrayid']] = thistime
        else:
            examples[row['arrayid']] = examples[row['arrayid']] + bestsofar
            iterations[row['arrayid']].append(len(examples[row['arrayid']]))
            times[row['arrayid']] = times[row['arrayid']] + thistime
            if rundata[row['arrayid']] == None:
                rundata[row['arrayid']] = stats
    
for arrayid in rundata:
    rundata[arrayid]['iterations'] = iterations[arrayid]
    #print(f"{sum(iterations[arrayid])} =?= {len(examples[arrayid])}")
    rundata[arrayid]['data'] = examples[arrayid]
    if len(times[arrayid]) > 0:
        rundata[arrayid]['times'] = (np.array(times[arrayid]) - times[arrayid][0])/1000/60

In [ ]:
legend = []
plt.figure(figsize=[12,9])
for item in rundata:
    results = pd.DataFrame(rundata[item]['data'], columns=['index', 'fitness', 'individual'])
    trend = [1] + list(results['fitness']);
    legend.append(f"{item}-{rundata[item]['input_file'].split('/')[-1][:-5]} ({trend[-1]})")

    plt.legend(legend)
    plt.plot(trend[:50])
    #plottrend(trend, iterations[item])

In [ ]:
item = 1
results = pd.DataFrame(rundata[item]['data'], columns=['index', 'fitness', 'individual'])

max = 3
for index,row in results.iterrows():
    if max < row['fitness']:
        print(f"ERROR index={row['index']}: {last['fitness']} < {row['fitness']}")
        print(f"  {last['individual']}")
        print(f"  {row['individual']}")
    print(row['fitness'])
    max = row['fitness']
    last = row.copy()

In [ ]:
item = 1
results = pd.DataFrame(rundata[item]['data'], columns=['index', 'fitness', 'individual'])
results
